In [1]:
import pandas as pd
import datetime

In [2]:
# mta_2011 = pd.read_csv("dataset/mta-data/MTA_Turnstile_2011.csv", sep=',', header=None, usecols=[1,3,4,6,7])
mta_2012 = pd.read_csv("dataset/mta-data/MTA_Turnstile_2012.csv", sep=',', header=None, usecols=[1,3,4,6,7])
# mta_2013 = pd.read_csv("dataset/mta-data/MTA_Turnstile_2013.csv", sep=',', header=None, usecols=[1,3,4,6,7])
# mta_2014_1 = pd.read_csv("dataset/mta-data/MTA_Turnstile_2014_1.csv", sep=',', header=None, usecols=[1,3,4,6,7])
# mta_2014_2 = pd.read_csv("dataset/mta-data/MTA_Turnstile_2014_2.csv", sep=',', header=None, usecols=[1,6,7,9,10])
# mta_2015 = pd.read_csv("dataset/mta-data/MTA_Turnstile_2015.csv", sep=',', header=None, usecols=[1,6,7,9,10])
# mta_2016 = pd.read_csv("dataset/mta-data/MTA_Turnstile_2016.csv", sep=',', header=None, usecols=[1,6,7,9,10])

location_data = pd.read_csv("dataset/Remote-Booth-Station.csv", sep=',', usecols=[0,2])
location_data = location_data.drop_duplicates()
location_data = location_data.drop_duplicates(['Remote'], keep='first')

weather_data = pd.read_csv("dataset/clean-weather-data/weather.csv", sep=',', header=None)
weather_data.columns = ['Date','Event']
weather_data = weather_data.fillna('Normal')

C:\Users\Victor\Anaconda3\envs\python2\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# print mta_2011.shape
print mta_2012.shape
# print mta_2013.shape
# print mta_2014_1.shape
# print mta_2014_2.shape
# print mta_2015.shape
# print mta_2016.shape

(11015011, 5)


In [19]:
def convert_datetime(dt):
    for fmt in ('%m-%d-%y', '%Y-%m-%d'):
        try:
            return datetime.datetime.strptime(dt, fmt).strftime('%m/%d/%Y')
        except ValueError:
            pass

def cleanOld(mta_data, weather_data, location_data):
    mta_data.columns = ["Remote", "Date", "Hour", "Entry", "Exit"]
    
    # Fixing formats and removing unwanted dates
    mta_data = mta_data.dropna(how='any')
    mta_data['Date'] = mta_data['Date'].astype('str')
    mta_data = mta_data.loc[(mta_data['Date'].str.contains("-") == True) & (mta_data['Exit'].str.contains("\r\n") == False)]
    mta_data['Entry'] = mta_data['Entry'].astype('float')
    mta_data['Exit'] = mta_data['Exit'].astype('float')
    
    mta_data['Date'] = mta_data['Date'].apply(lambda x: convert_datetime(x))
    mta_data = mta_data.merge(location_data,how='inner',on='Remote')
    mta_data = mta_data.merge(weather_data,how='inner',on='Date')
    mta_data = mta_data.set_index(['Remote','Date','Hour','Station','Event']) \
                    .diff() \
                    .fillna(float(0)) \
                    .reset_index()
    mta_data.loc[(mta_data.Entry < 0.0) | (mta_data.Entry > 10000.0), ['Entry','Exit']] = float(0)
    return mta_data
    
def cleanNew(mta_data, weather_data, location_data):
    mta_data.columns = ["Remote", "Date", "Hour", "Entry", "Exit"]
    
    # Fixing formats
    mta_data['Entry'] = mta_data['Entry'].astype('float')
    mta_data['Exit'] = mta_data['Exit'].astype('float')
    
    mta_data = mta_data.merge(location_data,how='inner',on='Remote')
    mta_data = mta_data.merge(weather_data,how='inner',on='Date')
    mta_data = mta_data.set_index(['Remote','Date','Hour','Station','Event']) \
                    .diff() \
                    .fillna(float(0)) \
                    .reset_index()
    mta_data.loc[(mta_data.Entry < 0.0) | (mta_data.Entry > 10000.0), ['Entry','Exit']] = float(0)
    return mta_data

In [20]:
# print 'Starting on year 2011...'
# y2011 = cleanOld(mta_2011, weather_data, location_data)
print 'Starting on year 2012...'
y2012 = cleanOld(mta_2012, weather_data, location_data)
# print 'Starting on year 2013...'
# y2013 = cleanOld(mta_2013, weather_data, location_data)
# print 'Starting on year 2014 (1)...'
# y2014_1 = cleanOld(mta_2014_1, weather_data, location_data)
# print 'Starting on year 2014 (2)...'
# y2014_2 = cleanNew(mta_2014_2, weather_data, location_data)
# print 'Starting on year 2015...'
# y2015 = cleanNew(mta_2015, weather_data, location_data)
# print 'Starting on year 2016...'
# y2016 = cleanNew(mta_2016, weather_data, location_data)

Starting on year 2012...


C:\Users\Victor\Anaconda3\envs\python2\lib\site-packages\ipykernel\__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [21]:
y2012.head()

,Remote,Date,Hour,Station,Event,Entry,Exit
0,R387,04/24/2012,17:00:00,E TREMONT AVE,Normal,0.0,0.0
1,R387,04/24/2012,21:00:00,E TREMONT AVE,Normal,0.0,0.0
2,R387,04/24/2012,01:00:00,E TREMONT AVE,Normal,0.0,0.0
3,R387,04/24/2012,05:00:00,E TREMONT AVE,Normal,0.0,0.0
4,R387,04/24/2012,09:00:00,E TREMONT AVE,Normal,0.0,0.0


In [22]:
print y2012.shape

(261924, 7)


In [23]:
y2012[200000:200100]

,Remote,Date,Hour,Station,Event,Entry,Exit
200000,R057,06/30/2012,16:00:00,ATLANTIC AVE,Normal,563.0,70.0
200001,R057,06/30/2012,20:00:00,ATLANTIC AVE,Normal,602.0,87.0
200002,R057,06/30/2012,00:00:00,ATLANTIC AVE,Normal,0.0,0.0
200003,R057,06/30/2012,04:00:00,ATLANTIC AVE,Normal,44.0,7.0
200004,R057,06/30/2012,04:00:00,ATLANTIC AVE,Normal,0.0,0.0
200005,R057,06/30/2012,08:00:00,ATLANTIC AVE,Normal,66.0,25.0
200006,R057,06/30/2012,12:00:00,ATLANTIC AVE,Normal,299.0,66.0
200007,R057,06/30/2012,16:00:00,ATLANTIC AVE,Normal,551.0,97.0
200008,R057,06/30/2012,20:00:00,ATLANTIC AVE,Normal,522.0,92.0
200009,R057,06/30/2012,00:00:00,ATLANTIC AVE,Normal,0.0,0.0


In [ ]:
# print 'Concatenating all the data...'
# completeData = pd.concat([y2011,y2012,y2013,y2014_1,y2014_2,y2015,y2016])
# print completeData.shape

In [ ]:
# completeData.head()